<a href="https://colab.research.google.com/github/rahulsm27/ML/blob/main/Pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
# Installing required libraries
!pip install torchinfo
import torchinfo
import torch
from torch import nn,optim
import torch.nn.functional as F
import pandas as pd
import numpy as np

In [3]:
#device = "cuda" if torch.cuda.is_available() else "cpu"
device = "cpu"

In [4]:
# Creating our own LeNet5

class LeNet5(nn.Module):
  def __init__(self):
    super(LeNet5,self).__init__()
    self.conv1 = nn.Conv2d(3,6,5) # in channel , out channe, kernel
    self.relu1 = nn.ReLU()
    self.maxpool1 = nn.MaxPool2d((2,2))

    self.conv2 = nn.Conv2d(6,16,5)  # in channel , out channe, kernel
    self.relu2 = nn.ReLU()
    self.maxpool2 = nn.MaxPool2d((2,2))

    self.fc1 = nn.Linear(16*5*5,120)
    self.fc2 = nn.Linear(120,84)
    self.fc3 = nn.Linear(84,10)

  def forward(self, x):
    # x = F.max_pool2d(F.relu(self.conv1(x)),(2, 2))
    # x = F.max_pool2d(F.relu(self.conv2(x)), 2)
    x = self.conv1(x)
    x = self.relu1(x)
    x= self.maxpool1(x)

    x = self.conv2(x)
    x = self.relu2(x)
    x= self.maxpool2(x)

    x = x.view(-1,int(x.nelement() / x.shape[0]))

    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = self.fc3(x)



    return x



model = LeNet5().to(device)

In [5]:
from torchvision import transforms
from torchvision.datasets import CIFAR10
train_transforms = transforms.Compose([ transforms.RandomCrop(32, padding=4), transforms.RandomHorizontalFlip(), transforms.ToTensor(), transforms.Normalize(mean=(0.4914, 0.4822, 0.4465), std=(0.2023, 0.1994, 0.2010))])
train_data = CIFAR10(root="./train/", train=True, download=True, transform=train_transforms)
trainloader = torch.utils.data.DataLoader( train_data,batch_size=16, shuffle=True)

100%|██████████| 170498071/170498071 [00:12<00:00, 13373549.09it/s]


Extracting ./train/cifar-10-python.tar.gz to ./train/


In [6]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(),lr=0.001, momentum=0.9)

In [7]:
N_EPOCHS = 20
for epoch in range(N_EPOCHS):
  epoch_loss = 0.0
  for inputs, labels in trainloader:
    inputs = inputs.to(device)
    labels = labels.to(device)

    optimizer.zero_grad()

    outputs = model(inputs)

    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()
    epoch_loss += loss.item()
  print("Epoch: {} Loss: {}".format(epoch,epoch_loss/len(trainloader)))

Epoch: 0 Loss: 1.9791963856887818
Epoch: 1 Loss: 1.635020360546112
Epoch: 2 Loss: 1.4983821289634704
Epoch: 3 Loss: 1.4150109517097473
Epoch: 4 Loss: 1.3551741639900208
Epoch: 5 Loss: 1.3074513339805602
Epoch: 6 Loss: 1.2631094504928588
Epoch: 7 Loss: 1.2307385359191894
Epoch: 8 Loss: 1.2021612531375885
Epoch: 9 Loss: 1.1819142775344849
Epoch: 10 Loss: 1.1565085112285614
Epoch: 11 Loss: 1.142551529378891
Epoch: 12 Loss: 1.126042156085968
Epoch: 13 Loss: 1.1133582607269288
Epoch: 14 Loss: 1.0945655228996276
Epoch: 15 Loss: 1.0882967677211761
Epoch: 16 Loss: 1.076190096540451
Epoch: 17 Loss: 1.0695812685489654
Epoch: 18 Loss: 1.0549307771396637
Epoch: 19 Loss: 1.0451562338352203


In [14]:
y_pred = model(train_data[1][0].unsqueeze(0).to(device))
print(f" Model predicted label : {np.argmax(y_pred.detach().numpy())}")
print(f" Actual label : {train_data[1][1]}")

 Model predicted label : 9
 Actual label : 9


In [24]:
import numpy as np

def softmax(z):
    e_z = np.exp(z)
    return e_z / e_z.sum()

# Example usage:
input_vector = np.array([2.0, 1.0, 0.1])
softmax_probs = softmax(input_vector)
print(softmax_probs)


[0.65900114 0.24243297 0.09856589]


In [23]:
softmax_log = np.log(softmax_probs) * -1
print(softmax_log)

[0.41703002 1.41703002 2.31703002]
